In [0]:
df = spark.read.json("/Volumes/workspace/default/pokemon_volume/pokemon/json/")
df.createOrReplaceTempView("pokemons")
display(df)

In [0]:
query = '''
    SELECT date_ref, poke_info.*
    FROM pokemons
    LATERAL VIEW EXPLODE(results) as poke_info
    QUALIFY row_number() OVER(PARTITION BY poke_info.name ORDER BY date_ref DESC) = 1
'''

df = spark.sql(query).coalesce(1)
(df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("workspace.bronze.pokedex"))

In [0]:
df.display()